## 1.Data Preprocessing

### Cleaning + Estimating volatility
- Ensuring validity of datapoints 
- Cleaning out deviations | Isolated points | etc ...
- Estimating Volatility

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from datetime import datetime

df = pd.read_csv("/Users/aleksandr/Desktop/Meta_Test.csv")
df = df.dropna()

In [2]:
from preprocess_td import preprocess_tick_data

# Would be lovely to estimate parameters of function
df_clean, df_diagnostics, outlier_counter = preprocess_tick_data(df)
df = df_clean
df = df.drop(columns="VOLATILITY")

Starting preprocessing with 570771 rows
After filtering trading hours: 282810 rows


KeyboardInterrupt: 

In [3]:
# Would be lovely to estimate parameters of function
from volatility_estimation import estimate_tick_volatility

df = estimate_tick_volatility(df, method = 'wavelet')

Estimating advanced tick-level volatility for 278585 ticks...
Computing wavelet-based volatility for META.O...
Completed advanced tick-level volatility estimation


In [4]:
df.drop(columns=['return', "SYMBOL"], inplace= True)
df.rename(columns={'wavelet_vol' : 'Volatility', 
                  'TIMESTAMP':'Timestamp',
                   'VALUE' : 'Value',
                   'VOLUME' : 'Volume'}, inplace=True)
df.head()

,Timestamp,Value,Volume,Volatility
0,2025-01-30 09:30:00.740000+00:00,694.24,13.0,0.000260
1,2025-01-30 09:30:00.740000+00:00,694.17,15.0,0.000260
2,2025-01-30 09:30:00.740000+00:00,694.17,15.0,0.000261
3,2025-01-30 09:30:00.740000+00:00,694.11,8.0,0.000261
4,2025-01-30 09:30:00.740000+00:00,694.10,249.0,0.000261


## 2. Synthetic Noise Injection 

Suppose that, the latent log-price Xt is an Ito-semimartingale of the form 

$dX_t = b_t dt + \sigma_t dW_t + dJ_t,$  
$d\sigma_t = \tilde{b}_t dt + \tilde{\sigma}^{(1)}_t dW_t + \tilde{\sigma}^{(2)}_t d\tilde{W}_t + d\tilde{J}_t$





### Market Microstructure Noise:
- Quote Spread Noise: The price fluctuation caused by trades alternating between bid and ask prices, creating a "bouncing" effect that obscures the true efficient price.

- Order Flow Noise: Price movements driven by the imbalance between buy and sell orders, where persistent directional trading pressure can create temporary price trends away from the efficient price.

- Strategic Order Noise: Price distortions created when large traders split their orders into smaller pieces to minimize market impact.

- Quote Positioning Noise: Price effects from market makers strategically placing and canceling quotes to create false impressions of supply and demand.

In [7]:
# Generate 1000 noise samples
from microstructure_simulator import MarketMicrostructureSimulator

simulator = MarketMicrostructureSimulator(n_samples=1000)
result = simulator.simulate_full_microstructure()
noise_components = simulator.extract_noise_components(result)
noise_components["total_noise"][:10]

Simulating efficient price process...
Adding quote spread noise...
Simulating order flow noise...
Simulating strategic order splitting...
Simulating strategic quote positioning...


array([-0.03378606,  0.03567945,  0.09798677,  0.10495182, -0.12444308,
       -0.01157954, -0.20460404,  0.04836754, -0.03279175,  0.04115321])

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from datetime import datetime
from preprocess_td import preprocess_tick_data
from volatility_estimation import estimate_tick_volatility

df = pd.read_csv("/Users/aleksandr/Desktop/Meta_Test.csv")
df = df.dropna()

# Would be lovely to estimate parameters of function
df_clean, df_diagnostics, outlier_counter = preprocess_tick_data(df)
df = df_clean
df = df.drop(columns="VOLATILITY")

df = estimate_tick_volatility(df, method = 'wavelet')

df.drop(columns=['return', "SYMBOL"], inplace= True)
df.rename(columns={'wavelet_vol' : 'Volatility', 
                  'TIMESTAMP':'Timestamp',
                   'VALUE' : 'Value',
                   'VOLUME' : 'Volume'}, inplace=True)
df.head()

Starting preprocessing with 570771 rows
After filtering trading hours: 282810 rows
After cleaning outliers: 282301 rows
Final clean dataset: 278585 rows

Outlier counts by detection method:
  zscore: 64
  extreme_deviation: 69
  isolated_point: 390
  price_reversal: 93
  market_open_artifact: 0
  timestamp_group: 34
  price_velocity: 3703
  suspicious_cluster: 52
  wavelet_outlier: 24
Estimating advanced tick-level volatility for 278585 ticks...
Computing wavelet-based volatility for META.O...
Completed advanced tick-level volatility estimation


,Timestamp,Value,Volume,Volatility
0,2025-01-30 09:30:00.740000+00:00,694.24,13.0,0.000260
1,2025-01-30 09:30:00.740000+00:00,694.17,15.0,0.000260
2,2025-01-30 09:30:00.740000+00:00,694.17,15.0,0.000261
3,2025-01-30 09:30:00.740000+00:00,694.11,8.0,0.000261
4,2025-01-30 09:30:00.740000+00:00,694.10,249.0,0.000261


In [1]:
file_path = "/Users/aleksandr/Desktop" + "/my_data.csv"
df_sample = df[:2000]
# Save DataFrame to CSV
df_sample.to_csv(file_path, index=False) 

NameError: name 'df' is not defined

In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv("/Users/aleksandr/Desktop/my_data.csv")

In [5]:
df = df[:2000]
df.shape

(2000, 4)

In [6]:
from volatility_regimes_identification.tda_volatility_pipeline import TDAVolatilityPipeline

pipeline = TDAVolatilityPipeline()

# Process entire dataset
result_df = pipeline.process_data(
    df,
    timestamp_col='Timestamp',
    price_col='Value',
    volume_col='Volume',
    volatility_col='Volatility',
    n_regimes=4,
    training_batch=1700,  
    batch_size=100       
)

TDA Volatility Pipeline initialized

Starting TDA volatility regime pipeline at 20:50:08

Using first 1,700 observations for TDA analysis
Extracting microstructure features with window sizes: [10, 50, 100]
Computing microstructure features...
Microstructure features computed in 0.01 seconds
Computing order flow metrics...
Order flow metrics computed in 0.00 seconds
Computing DMI features...
DMI features computed in 0.01 seconds
Extracted 45 features from 1700 observations
Feature extraction time: 0.05 seconds

=== Using Progressive Feature Selection: 45 features ===
Computing initial feature relevance scores...
Initial scoring completed in 1.02 seconds
Top 5 features by initial score:
  adx_100: 0.1000
  adx_50: 0.0941
  sin_time: 0.0939
  tr_100: 0.0939
  tr_50: 0.0930
First-stage selection: 30 candidates from 45 features
Selected 30 features from 45
Selected features: ['Value', 'seconds_from_open', 'sin_time', 'cos_time', 'trade_volume_profile_10', 'trade_frequency_10', 'tick_imbalan

In [4]:
result_df.shape

(500, 6)

In [7]:
# Get the regime values as a list
regimes = result_df["regime"].tolist()

# Initialize counters
current_regime = regimes[0]
current_count = 1
regime_sequences = {}

# Count sequential regimes
for i in range(1, len(regimes)):
    if regimes[i] == current_regime:
        current_count += 1
    else:
        # Store the sequence count
        key = f"Regime {current_regime}"
        if key not in regime_sequences:
            regime_sequences[key] = []
        regime_sequences[key].append(current_count)
        
        # Reset counters
        current_regime = regimes[i]
        current_count = 1

# Add the last sequence
key = f"Regime {current_regime}"
if key not in regime_sequences:
    regime_sequences[key] = []
regime_sequences[key].append(current_count)

# Print results
print("Sequential regime counts:")
for regime, sequences in regime_sequences.items():
    print(f"\n{regime}:")
    print(f"Number of sequences: {len(sequences)}")
    print(f"Sequence lengths: {sequences}")
    print(f"Average sequence length: {sum(sequences)/len(sequences):.2f}")

Sequential regime counts:

Regime 1.0:
Number of sequences: 30
Sequence lengths: [10, 30, 1, 4, 465, 119, 1, 1, 125, 5, 5, 37, 1, 128, 10, 2, 3, 1, 2, 1, 2, 7, 3, 65, 2, 2, 1, 71, 11, 64]
Average sequence length: 39.30

Regime 0.0:
Number of sequences: 10
Sequence lengths: [34, 1, 2, 10, 49, 66, 2, 3, 1, 5]
Average sequence length: 17.30

Regime 3.0:
Number of sequences: 13
Sequence lengths: [24, 19, 33, 3, 1, 28, 87, 46, 1, 23, 67, 1, 1]
Average sequence length: 25.69

Regime 2.0:
Number of sequences: 10
Sequence lengths: [30, 22, 56, 159, 3, 1, 1, 2, 20, 20]
Average sequence length: 31.40


## 3. Model Architecture Adaptations

Modify the original CSDI code (GitHub) for denoising:

### Remove Imputation Logic:
- Delete code blocks that handle missing value imputation.

### Time Embeddings:
- Encode irregular timestamps as sinusoidal embeddings (normalized to [0,1]).

### Conditioning Mechanism:
- Use the noisy input as the conditional context (instead of partial observations).

### Diffusion Process:
- Use the original diffusion steps but disable masking (all positions are observed).
- Adjust the noise schedule (β) to match financial noise characteristics.


## 4. Training Pipeline

**Objective:** Learn to reverse the diffusion process conditioned on noisy ticks.

### Inputs:
- **noisy_data:** Corrupted ticks (price, volume, etc.).
- **mask:** All ones (no missing data).
- **time_embeddings:** Encoded timestamps.

### Forward Process:
- Gradually add Gaussian noise to `noisy_data` across diffusion timesteps.
- Forward Process: Replace Gaussian SDE with a market-realistic stochastic process.

### Reverse Process:
- Train the model to predict the score (gradient) to denoise the data.

### Loss Function:
- Weighted MSE between predicted and true noise at each diffusion step.

## 5. Key Functions to Implement

### Data Loader:
```python
def load_tick_data():
    """Reads raw ticks and converts to windowed sequences."""
    pass

def inject_microstructure_noise():
    """Adds synthetic bid-ask bounce, order flow noise."""
    pass
```

### Time Embeddings:
```python
def encode_timestamps():
    """Converts irregular timestamps to continuous embeddings."""
    pass
```

### Diffusion Utils:
```python
def beta_scheduler():
    """Defines the noise schedule (linear, cosine, etc.)."""
    pass

def q_sample():
    """Forward diffusion process (adding noise)."""
    pass
```

### Model:
```python
class ConditionalScoreModel:
    """Modified CSDI backbone (transformer/TCN)."""
    pass

def train_step():
    """Computes loss and updates weights."""
    pass
```

## 6. Training Process

### Hyperparameters:
- Diffusion steps (`T=1000`), learning rate (`1e-4`), batch size (`64`).
- Noise schedule (e.g., `beta_start=1e-4`, `beta_end=0.02`).

### Training Loop:
For each batch:
1. Generate noisy data via `inject_microstructure_noise()`.
2. Compute time embeddings for irregular timestamps.
3. **Forward pass:** Corrupt noisy data with diffusion.
4. **Reverse pass:** Predict denoised data.
5. Update model weights via gradient descent.

### Checkpointing:
- Save model weights periodically (e.g., every epoch).
- Track validation loss on a held-out tick dataset.

## 7. Validation & Testing

### Metrics:
- **Reconstruction Loss:** MSE between denoised and clean data (if synthetic).
- **Volatility Consistency:** Compare realized volatility of raw vs. denoised data.
- **Microstructure Preservation:** Autocorrelation of trade signs.

### Visual Checks:
- Plot raw vs. denoised ticks (ensure no new timestamps are added).

## 8. Deployment Pipeline

### Preprocessing:
- Normalize new tick data using the same scaler from training.
- Encode timestamps.

### Inference:
- Run the trained CSDI in reverse diffusion mode with `mask=1`.

### Postprocessing:
- Inverse-transform normalized denoised data.